## 1. 导入库

In [71]:
import sklearn

In [72]:
sklearn.__version__

'0.19.2'

In [73]:
# https://blog.csdn.net/xiaosa_kun/article/details/84868437

In [74]:
import sys
sys.path.append("/Users/zhengwenjie/AI/work/ML_3/2017-CCF-BDCI-AIJudge")
import pandas as pd
from config.db_config import Config
import numpy as np
from utils import LOGGER

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
import codecs

## 2.加载数据

In [75]:
config = Config()
config.data_csv_path = "/Users/zhengwenjie/AI/work/ML_3/2017-CCF-BDCI-AIJudge/data/output/corpus/data1.csv"

In [76]:
train_df = pd.read_csv(config.data_csv_path,encoding='utf-8',sep=',')
train_df = train_df.dropna()
train_df['penalty'] = train_df['penalty'].astype(int)
train_df.head()

,id,content,laws,penalty
0,16,公诉 机关 霍邱县 人民检察院 被告人 许某 1975 日生 2012 因涉嫌 危险 驾驶 ...,"133,72,73",3
1,32,公诉 机关 海口市 龙华区 人民检察院 被告人 王某 海口市 龙华区 人民检察院 海龙 检公...,"347,67,52,64",1
2,41,公诉 机关 广东省 潮州市 人民检察院 被告人 覃学彬 1980 出生 广西壮族自治区 大新...,"263,25,52,53,55,56",5
3,57,公诉 机关 榆林市 榆阳区 人民检察院 上诉人 原审 被告人 2012 因涉嫌 盗窃罪 榆林...,"264,52,53,67,72,73",5
4,60,公诉 机关 榆阳区 人民检察院 上诉人 原审 被告人 刘某 汉族 陕西省 横山县 小学文化 ...,"224,25,26,27,52,72",7


In [77]:
train_df.shape

(999, 4)

## 查看训练集每个分类的名字以及样本数量

In [78]:
# 类别：数据不均衡，需要进行采样 to do
for name,group in train_df.groupby(by='penalty'):
    print(name,len(group))

1 135
2 170
3 93
4 54
5 105
6 148
7 283
8 11


In [79]:
words_list=[]

In [80]:
for document in train_df['content']:
    words_list.append(document.split())

In [81]:
print(words_list[0:2])

[['公诉', '机关', '霍邱县', '人民检察院', '被告人', '许某', '1975', '日生', '2012', '因涉嫌', '危险', '驾驶', '霍邱县', '公安局', '取保候审', '2013', '日经', '本院', '取保候审', '霍邱县', '人民检察院', '以霍检', '刑诉', '2013', '起诉书', '指控', '被告人', '许某', '甲犯', '危险', '驾驶', '2013', '日向', '本院', '提起公诉', '本院', '依法', '简易程序', '实行', '独任', '审判', '2013', '公开', '开庭审理', '本案', '霍邱县', '人民检察院', '检察员', '胡涛', '被告人', '许某', '到庭', '参加', '诉讼', '现已', '审理', '终结', '霍邱县', '人民检察院', '指控', '2012', '被告人', '许某', '酒后', '驾驶', '二轮', '摩托车', '沿霍寿路', '由南向北', '行驶', '霍寿路', '公园路', '交叉口', '路边', '行人', '相撞', '公安民警', '查获', '六安市', '疾病', '预防', '控制中心', '鉴定', '许某', '血液', '乙醇', '含量', '169.64', 'mg', '100ml', '上述事实', '被告人', '开庭审理', '过程', '无异议', '被害人', '杨正响', '陈述', '证人', '李某', '证言', '六安市', '疾病', '预防', '控制中心', '检验', '报告', '六安市', '疾控交', '检字', '2012', '155', '霍邱县', '公安局', '交通管理', '大队', '呼吸', '酒精', '检测', '抽取', '当事人', '血样', '登记表', '驾驶', '信息', '查询', '道路', '交通事故', '赔偿', '调解', '协议书', '经济', '赔偿', '凭证', '谅解', '被告人', '户籍', '信息', '证据', '证明', '足以认定'], ['公诉', '机关', '海口市', '龙华区', '人民检察院', '被告人', '王某', '海口市

## 3. 特征工程
TfidfVectorizer模型

In [82]:
#调用sklearn.feature_extraction.text库的TfidfVectorizer方法实例化模型对象。
#TfidfVectorizer方法需要4个参数。
#第1个参数是分词结果，数据类型为列表，其中的元素也为列表；
#第2个关键字参数stop_words是停顿词，数据类型为列表；
#第3个关键字参数min_df是词频低于此值则忽略，数据类型为int或float;
#第4个关键字参数max_df是词频高于此值则忽略，数据类型为Int或float。
#查看TfidfVectorizer方法的更多参数用法，
#官方文档链接：http://sklearn.apachecn.org/cn/0.19.0/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [83]:
tfidf=TfidfVectorizer(words_list,min_df=3,max_df=0.95)

In [84]:
#第1行代码查看向量化的维数，即特征的维数；
#第2行代码调用TfidfVectorizer对象的fit_transform方法获得特征矩阵赋值给X；
#第3行代码查看特征矩阵的形状。
X=tfidf.fit_transform(train_df['content'])
print('词表大小:', len(tfidf.vocabulary_))
print(X.shape)

词表大小: 5983
(999, 5983)


In [85]:
#tfidf.vocabulary_

In [86]:
X.toarray()[0:2]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## 4. 模型训练

### 4.1 标签编码

In [87]:
np.unique(train_df['penalty'])

array([1, 2, 3, 4, 5, 6, 7, 8])

In [88]:
#调用sklearn.preprocessing库的LabelEncoder方法对文章分类做标签编码。
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(train_df['penalty'])

In [89]:
y.shape

(999,)

In [90]:
y

array([2, 0, 4, 4, 6, 0, 6, 1, 1, 1, 3, 7, 1, 7, 4, 1, 5, 0, 6, 6, 6, 1,
       6, 2, 3, 1, 1, 6, 7, 1, 2, 6, 0, 6, 1, 3, 1, 6, 0, 0, 1, 1, 6, 2,
       6, 3, 1, 2, 5, 6, 0, 1, 6, 4, 1, 6, 1, 1, 0, 0, 1, 1, 3, 5, 1, 4,
       1, 0, 5, 2, 0, 1, 0, 2, 4, 0, 5, 4, 6, 6, 5, 6, 6, 0, 4, 2, 1, 6,
       6, 3, 2, 1, 6, 0, 0, 6, 6, 0, 3, 0, 1, 1, 1, 1, 0, 0, 1, 6, 1, 5,
       2, 1, 1, 1, 2, 2, 6, 1, 1, 1, 0, 3, 0, 2, 5, 5, 6, 4, 6, 5, 1, 4,
       1, 7, 2, 3, 6, 2, 6, 2, 6, 3, 3, 6, 4, 0, 1, 1, 6, 2, 3, 5, 6, 3,
       1, 6, 2, 6, 1, 6, 6, 2, 0, 4, 0, 0, 0, 0, 2, 0, 0, 5, 0, 0, 0, 0,
       6, 0, 6, 1, 0, 2, 0, 0, 6, 4, 3, 0, 4, 6, 6, 6, 4, 6, 5, 6, 4, 6,
       5, 6, 4, 5, 4, 3, 2, 1, 0, 5, 1, 6, 5, 6, 5, 5, 1, 6, 6, 6, 5, 6,
       5, 5, 6, 0, 6, 5, 4, 6, 6, 6, 5, 4, 6, 6, 5, 5, 5, 1, 6, 7, 5, 1,
       5, 6, 5, 4, 6, 1, 5, 3, 3, 4, 4, 1, 0, 3, 5, 1, 0, 1, 4, 1, 1, 3,
       4, 4, 4, 6, 6, 6, 4, 1, 1, 5, 2, 6, 0, 0, 4, 1, 5, 1, 1, 1, 4, 0,
       1, 1, 6, 1, 0, 0, 5, 0, 0, 1, 6, 6, 5, 5, 1,

### 4.2 逻辑回归模型
调用sklearn.linear_model库的LogisticRegression方法实例化模型对象。  <br>
调用sklearn.model_selection库的train_test_split方法划分训练集和测试集。

In [91]:
def micro_avg_f1(y_true, y_pred):
    '''
    分类 评估函数
    :param y_true: 样本实际类别
    :param y_pred: 预测类别
    :return:
    '''
    return f1_score(y_true, y_pred, average='micro')

In [92]:
X_train,X_test,y_train,y_test= train_test_split(X, y, test_size=0.3)

In [93]:
lr = LogisticRegression(multi_class='multinomial', solver='lbfgs')
lr.fit(X_train, y_train)
y_pred = lr.predict_proba(X_test)
y_pred_score = micro_avg_f1(y_test,lr.predict(X_test))
y_pred_score

0.3933333333333333

In [94]:
y_pred

array([[0.09196838, 0.10231541, 0.09985797, ..., 0.21166389, 0.30181226,
        0.01678982],
       [0.12568477, 0.18536485, 0.07682697, ..., 0.25146699, 0.20635935,
        0.01314593],
       [0.04800576, 0.05904625, 0.04165406, ..., 0.14125287, 0.58440859,
        0.01554025],
       ...,
       [0.07192499, 0.30269189, 0.16065814, ..., 0.16715899, 0.11183075,
        0.01087268],
       [0.08215362, 0.12223264, 0.06115791, ..., 0.15924179, 0.38486067,
        0.01379508],
       [0.17194874, 0.10197139, 0.07179823, ..., 0.10298619, 0.33647239,
        0.01406117]])

In [95]:
lr.score(X_test,y_test)

0.3933333333333333

### 4. 3 交叉验证

In [96]:
skf = StratifiedKFold(n_splits=2, random_state=config.seed)

In [97]:
for train_index, test_index in skf.split(X_train, y_train):
    print("TRAIN:", train_index, "TEST:", test_index)
    train_data,val_data=X_train[train_index],X_train[test_index]
    train_y,val_y=y_train[train_index],y_train[test_index]
    print("Train_dataset...")
    print(train_data.toarray(),train_y)
    print("Validate_dataset...")
    print(val_data.toarray(),val_y)
    lr = LogisticRegression(multi_class='multinomial', solver='lbfgs')
    lr.fit(train_data, train_y)
    y_pred_val=lr.predict(val_data)
    print("y_true y_pred.....")
    print(val_y,y_pred_val)
    y_pred_score = micro_avg_f1(val_y,y_pred_val)
    y_pred_score
    print("y_pred_score={}".format(y_pred_score))

TRAIN: [312 318 320 329 335 336 339 341 343 345 349 350 351 353 355 357 359 363
 366 368 369 370 371 372 373 374 375 376 379 380 381 382 383 384 385 386
 387 388 389 390 391 392 393 394 395 396 397 398 399 400 401 402 403 404
 405 406 407 408 409 410 411 412 413 414 415 416 417 418 419 420 421 422
 423 424 425 426 427 428 429 430 431 432 433 434 435 436 437 438 439 440
 441 442 443 444 445 446 447 448 449 450 451 452 453 454 455 456 457 458
 459 460 461 462 463 464 465 466 467 468 469 470 471 472 473 474 475 476
 477 478 479 480 481 482 483 484 485 486 487 488 489 490 491 492 493 494
 495 496 497 498 499 500 501 502 503 504 505 506 507 508 509 510 511 512
 513 514 515 516 517 518 519 520 521 522 523 524 525 526 527 528 529 530
 531 532 533 534 535 536 537 538 539 540 541 542 543 544 545 546 547 548
 549 550 551 552 553 554 555 556 557 558 559 560 561 562 563 564 565 566
 567 568 569 570 571 572 573 574 575 576 577 578 579 580 581 582 583 584
 585 586 587 588 589 590 591 592 593 594 595

y_true y_pred.....
[0 0 0 5 5 0 2 5 5 0 3 5 3 2 0 7 0 5 6 6 5 4 4 6 5 4 2 5 0 0 6 6 6 4 4 4 6
 1 1 5 2 4 6 5 4 4 0 7 6 4 4 6 6 1 6 6 0 1 6 5 1 2 4 1 6 2 6 0 0 6 5 6 2 6
 5 0 6 1 6 2 1 0 5 5 1 6 6 4 1 6 1 7 1 5 6 6 2 4 2 6 6 2 1 4 6 5 5 1 5 5 6
 6 6 6 2 6 2 1 6 5 3 1 1 6 1 5 1 3 1 0 5 6 0 4 2 5 2 6 4 0 0 1 6 5 0 4 1 1
 1 1 3 6 5 5 2 6 6 7 1 6 6 6 2 4 1 6 2 3 4 6 0 2 4 6 6 5 6 6 2 3 1 0 5 6 0
 1 5 0 3 0 6 0 1 0 6 3 1 6 6 0 6 5 6 6 0 6 4 0 2 4 1 0 2 0 4 6 6 0 6 6 3 1
 1 0 0 0 0 3 0 5 5 1 5 2 2 6 1 4 6 6 6 5 6 5 1 6 5 2 6 4 6 5 3 3 0 2 2 6 1
 6 5 6 6 0 1 1 1 6 6 0 0 1 6 1 6 1 3 6 4 6 6 6 1 1 3 6 6 6 5 0 1 1 6 2 0 0
 4 6 1 1 6 5 1 6 4 1 2 2 4 6 3 0 5 6 6 0 4 4 4 6 1 3 1 6 6 5 4 0 4 2 0 6 4
 5 3 2 6 1 5 5 0 1 4 1 5 6 6 5] [1 0 6 6 6 1 6 6 6 6 6 6 1 6 6 6 6 0 6 6 6 6 6 6 5 6 6 6 6 0 6 6 6 6 6 5 6
 1 6 6 1 5 6 6 6 6 1 6 6 6 6 1 6 6 6 6 0 1 6 6 1 6 6 1 6 6 6 6 0 6 6 6 1 6
 6 6 6 1 6 6 0 1 6 6 6 6 6 6 6 6 0 6 6 6 6 6 6 1 6 6 6 1 6 6 6 1 6 6 5 6 6
 6 6 6 6 6 2 1 6 6 6 1 1 6 1 5 6 6 6 6 1 6 1 6 1 

## 4.4 基于模型的特征提取

tf-idf 提供的原始样本后，然后通过LR。。。等模型提取 特征。  通过多个模型提取特征，进行特征融合，然后再进行训练

In [98]:
num_class = len(np.unique(train_df['penalty']))
num_class

8

In [99]:
# 初始化stack ： 通过模型提取特征矩阵的数值
stack=np.zeros((X_train.shape[0],num_class))
stack

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [100]:
skf = StratifiedKFold(n_splits=2, random_state=config.seed)
for train_index, test_index in skf.split(X_train, y_train):
    print("TRAIN:", train_index, "TEST:", test_index)
    train_data,val_data=X_train[train_index],X_train[test_index]
    train_y,val_y=y_train[train_index],y_train[test_index]
    print("Train_dataset...")
    print(train_data.toarray(),train_y)
    print("Validate_dataset...")
    print(val_data.toarray(),val_y)
    lr = LogisticRegression(multi_class='multinomial', solver='lbfgs')
    lr.fit(train_data, train_y)
    y_pred_val=lr.predict(val_data)
    print("y_true y_pred.....")
    print(val_y,y_pred_val,np.unique(train_y))
    y_pred_score = micro_avg_f1(val_y,y_pred_val)
    y_pred_score
    print("y_pred_score={}".format(y_pred_score))
    
    print("y_pred_val_prob....")
    y_pred_val_prob = lr.predict_proba(val_data)
    print(y_pred_val_prob)
    stack[test_index]+=y_pred_val_prob

TRAIN: [312 318 320 329 335 336 339 341 343 345 349 350 351 353 355 357 359 363
 366 368 369 370 371 372 373 374 375 376 379 380 381 382 383 384 385 386
 387 388 389 390 391 392 393 394 395 396 397 398 399 400 401 402 403 404
 405 406 407 408 409 410 411 412 413 414 415 416 417 418 419 420 421 422
 423 424 425 426 427 428 429 430 431 432 433 434 435 436 437 438 439 440
 441 442 443 444 445 446 447 448 449 450 451 452 453 454 455 456 457 458
 459 460 461 462 463 464 465 466 467 468 469 470 471 472 473 474 475 476
 477 478 479 480 481 482 483 484 485 486 487 488 489 490 491 492 493 494
 495 496 497 498 499 500 501 502 503 504 505 506 507 508 509 510 511 512
 513 514 515 516 517 518 519 520 521 522 523 524 525 526 527 528 529 530
 531 532 533 534 535 536 537 538 539 540 541 542 543 544 545 546 547 548
 549 550 551 552 553 554 555 556 557 558 559 560 561 562 563 564 565 566
 567 568 569 570 571 572 573 574 575 576 577 578 579 580 581 582 583 584
 585 586 587 588 589 590 591 592 593 594 595

y_true y_pred.....
[0 0 0 5 5 0 2 5 5 0 3 5 3 2 0 7 0 5 6 6 5 4 4 6 5 4 2 5 0 0 6 6 6 4 4 4 6
 1 1 5 2 4 6 5 4 4 0 7 6 4 4 6 6 1 6 6 0 1 6 5 1 2 4 1 6 2 6 0 0 6 5 6 2 6
 5 0 6 1 6 2 1 0 5 5 1 6 6 4 1 6 1 7 1 5 6 6 2 4 2 6 6 2 1 4 6 5 5 1 5 5 6
 6 6 6 2 6 2 1 6 5 3 1 1 6 1 5 1 3 1 0 5 6 0 4 2 5 2 6 4 0 0 1 6 5 0 4 1 1
 1 1 3 6 5 5 2 6 6 7 1 6 6 6 2 4 1 6 2 3 4 6 0 2 4 6 6 5 6 6 2 3 1 0 5 6 0
 1 5 0 3 0 6 0 1 0 6 3 1 6 6 0 6 5 6 6 0 6 4 0 2 4 1 0 2 0 4 6 6 0 6 6 3 1
 1 0 0 0 0 3 0 5 5 1 5 2 2 6 1 4 6 6 6 5 6 5 1 6 5 2 6 4 6 5 3 3 0 2 2 6 1
 6 5 6 6 0 1 1 1 6 6 0 0 1 6 1 6 1 3 6 4 6 6 6 1 1 3 6 6 6 5 0 1 1 6 2 0 0
 4 6 1 1 6 5 1 6 4 1 2 2 4 6 3 0 5 6 6 0 4 4 4 6 1 3 1 6 6 5 4 0 4 2 0 6 4
 5 3 2 6 1 5 5 0 1 4 1 5 6 6 5] [1 0 6 6 6 1 6 6 6 6 6 6 1 6 6 6 6 0 6 6 6 6 6 6 5 6 6 6 6 0 6 6 6 6 6 5 6
 1 6 6 1 5 6 6 6 6 1 6 6 6 6 1 6 6 6 6 0 1 6 6 1 6 6 1 6 6 6 6 0 6 6 6 1 6
 6 6 6 1 6 6 0 1 6 6 6 6 6 6 6 6 0 6 6 6 6 6 6 1 6 6 6 1 6 6 6 1 6 6 5 6 6
 6 6 6 6 6 2 1 6 6 6 1 1 6 1 5 6 6 6 6 1 6 1 6 1 

In [102]:
# LR 抽取特征后的结果
stack

array([[0.12825526, 0.17573644, 0.114927  , ..., 0.14180766, 0.2600508 ,
        0.01188213],
       [0.17634215, 0.32402361, 0.10040967, ..., 0.09672943, 0.1327857 ,
        0.01128139],
       [0.17261816, 0.26716498, 0.07849486, ..., 0.16734973, 0.16189277,
        0.01185092],
       ...,
       [0.05908211, 0.06806086, 0.04491911, ..., 0.07785718, 0.64294884,
        0.01869136],
       [0.12649882, 0.12838156, 0.07752169, ..., 0.12441757, 0.36674726,
        0.01744608],
       [0.24554545, 0.10254547, 0.05857376, ..., 0.12502165, 0.29478477,
        0.01453054]])

In [71]:
#调用pickle库的dump方法保存模型，需要2个参数。
#第1个参数是保存的对象，可以为任意数据类型，因为有3个模型需要保存，所以下面代码第1个参数是字典。
#第2个参数是保存的文件对象，数据类型为_io.BufferedWriter

In [72]:
import pickle

In [75]:
with open('tfidf_lr.model','wb') as f:
    save = {
        'label_encoder' : label_encoder,
        'tfidfVectorizer' : tfidf,
        'lr' : lr
    }
    pickle.dump(save, f)